In [1]:
import dann_model as dann
import training_helper as th
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import os
import numpy as np
from PIL import Image
import torchvision.models as models
import copy
from torchvision.utils import save_image
import PIL
import time
from sklearn.manifold import TSNE

In [2]:
device = th.getCudaDevice(cudaNum = 1, torchSeed = 123)
batch_size = 128
n_epoch = 10
my_net = dann.CNNModel().to(device)
optimizer = optim.Adam(my_net.parameters(), lr=1e-3)
loss_class = torch.nn.NLLLoss().to(device)
loss_domain = torch.nn.NLLLoss().to(device)

Device used: cuda:1


In [3]:
img_transform = transforms.Compose([
    transforms.Resize(28),
    #transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

mnist_train = th.labelImgData(root='hw3_data/digits/mnistm/train', transform=img_transform)
mnist_test = th.labelImgData(root='hw3_data/digits/mnistm/test', transform=img_transform)
svhn_train = th.labelImgData(root='hw3_data/digits/svhn/train', transform=img_transform)
svhn_test = th.labelImgData(root='hw3_data/digits/svhn/test', transform=img_transform)
usps_train = th.labelImgData(root='hw3_data/digits/usps/train', transform=img_transform)
usps_test = th.labelImgData(root='hw3_data/digits/usps/test', transform=img_transform)

mnist_train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=0)
mnist_test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=0)
svhn_train_loader = DataLoader(svhn_train, batch_size=batch_size, shuffle=True, num_workers=0)
svhn_test_loader = DataLoader(svhn_test, batch_size=batch_size, shuffle=False, num_workers=0)
usps_train_loader = DataLoader(usps_train, batch_size=batch_size, shuffle=True, num_workers=0)
usps_test_loader = DataLoader(usps_test, batch_size=batch_size, shuffle=False, num_workers=0)

In [5]:
def test(dataloader):
    my_net.eval()
    correct = 0
    for batch_idx, (img, label) in enumerate(dataloader):
        img, label = img.to(device), label.to(device)
        output, _ = my_net(img)
        _, pred = torch.max(output, 1)
        correct += (pred == label).sum()
    my_net.train()
    return correct.item() / len(dataloader.dataset) 

def train(filename):
    source_train, source_test = source
    target_train, target_test = target
    best_acc = 0
    for epoch in range(n_epoch):
        len_dataloader = min(len(source_train), len(target_train))
        data_source_iter, data_target_iter = iter(source_train), iter(target_train)
        for batch_idx in range(len_dataloader):
            # calculate lambda
            p = float(batch_idx + epoch * len_dataloader) / n_epoch / len_dataloader
            alpha = 2. / (1. + np.exp(-10 * p)) - 1

            # training model using source data
            s_img, s_label = data_source_iter.next()
            s_img, s_label = s_img.to(device), s_label.to(device)
            my_net.zero_grad()
            domain_label = torch.zeros(len(s_label)).long().to(device)
            class_output, domain_output = my_net(s_img, alpha=alpha)
            err_s_label = loss_class(class_output, s_label)
            err_s_domain = loss_domain(domain_output, domain_label)

            # training model using target data
            t_img, _ = data_target_iter.next()
            t_img = t_img.to(device)
            domain_label = torch.ones(len(t_img)).long().to(device)
            _, domain_output = my_net(t_img, alpha=alpha)
            err_t_domain = loss_domain(domain_output, domain_label)
            err = err_t_domain + alpha*(err_s_domain + err_s_label)
            err.backward()
            optimizer.step()
            if batch_idx % max(1, int(len_dataloader/3)) == 0:
                print('Epoch: {}/{}\t[Iter: {}/{} ({}%)]\tSource Labeling Loss: {:.4f}\tSource Domain Loss: {:.4f}\tTarget Domain Loss: {:.4f}'.format(
                      epoch+1, n_epoch, batch_idx, len_dataloader, int(batch_idx*100/len_dataloader), err_s_label, err_s_domain, err_t_domain))
        target_acc = test(target_test)
        print('*****Source Accuracy: {:.4f}\n*****Target Accuracy: {:.4f}'.format(test(source_test), target_acc),'\n')
        if target_acc > best_acc:
            best_acc = target_acc
            th.saveModel(filename+'_best.pth', my_net, optimizer)
    th.saveModel(filename+'_final.pth', my_net, optimizer)

In [6]:
# usps->mnistm
source = usps_train_loader, usps_test_loader
target = mnist_train_loader, mnist_test_loader
filename = 'p3_usps->mnistm'
th.loadModel(filename+'_final.pth', my_net, optimizer)
train(filename)

model loaded from p3_usps->mnistm_final.pth
Epoch: 1/10	[Iter: 0/57 (0%)]	Source Labeling Loss: 0.1820	Source Domain Loss: 0.5277	Target Domain Loss: 0.5619
Epoch: 1/10	[Iter: 19/57 (33%)]	Source Labeling Loss: 0.2432	Source Domain Loss: 2.4272	Target Domain Loss: 0.0636
Epoch: 1/10	[Iter: 38/57 (66%)]	Source Labeling Loss: 0.1712	Source Domain Loss: 1.2271	Target Domain Loss: 0.1793
*****Source Accuracy: 0.9377
*****Target Accuracy: 0.4359 

model saved to p3_usps->mnistm_best.pth
Epoch: 2/10	[Iter: 0/57 (0%)]	Source Labeling Loss: 0.1773	Source Domain Loss: 0.7919	Target Domain Loss: 0.3088
Epoch: 2/10	[Iter: 19/57 (33%)]	Source Labeling Loss: 0.1979	Source Domain Loss: 0.6969	Target Domain Loss: 0.3526
Epoch: 2/10	[Iter: 38/57 (66%)]	Source Labeling Loss: 0.2536	Source Domain Loss: 0.6694	Target Domain Loss: 0.4057
*****Source Accuracy: 0.9402
*****Target Accuracy: 0.4132 

Epoch: 3/10	[Iter: 0/57 (0%)]	Source Labeling Loss: 0.1908	Source Domain Loss: 0.5541	Target Domain Loss: 0.41

In [6]:
# mnistm->svhn
source = mnist_train_loader, mnist_test_loader
target = svhn_train_loader, svhn_test_loader
train('p3_mnistm->svhn')

Epoch: 1/10	[Iter: 0/469 (0%)]	Source Labeling Loss: 3.1088	Source Domain Loss: 0.9902	Target Domain Loss: 0.6395
Epoch: 1/10	[Iter: 156/469 (33%)]	Source Labeling Loss: 0.9300	Source Domain Loss: 1.8669	Target Domain Loss: 0.1478
Epoch: 1/10	[Iter: 312/469 (66%)]	Source Labeling Loss: 0.5946	Source Domain Loss: 1.3152	Target Domain Loss: 0.2472
Epoch: 1/10	[Iter: 468/469 (99%)]	Source Labeling Loss: 0.3819	Source Domain Loss: 0.9883	Target Domain Loss: 0.3356
*****Source Accuracy: 0.7733
*****Target Accuracy: 0.3099 

model saved to p3_mnistm->svhn_best.pth
Epoch: 2/10	[Iter: 0/469 (0%)]	Source Labeling Loss: 0.6323	Source Domain Loss: 0.9687	Target Domain Loss: 0.3694
Epoch: 2/10	[Iter: 156/469 (33%)]	Source Labeling Loss: 0.4383	Source Domain Loss: 0.9164	Target Domain Loss: 0.4291
Epoch: 2/10	[Iter: 312/469 (66%)]	Source Labeling Loss: 0.4907	Source Domain Loss: 0.7610	Target Domain Loss: 0.4683
Epoch: 2/10	[Iter: 468/469 (99%)]	Source Labeling Loss: 0.4586	Source Domain Loss: 0.73

In [10]:
# svhn->usps
source = svhn_train_loader, svhn_test_loader
target = usps_train_loader, usps_test_loader
train('p3_svhn->usps')

Epoch: 1/10	[Iter: 0/57 (0%)]	Source Labeling Loss: 2.3314	Source Domain Loss: 0.8420	Target Domain Loss: 0.6177
Epoch: 1/10	[Iter: 19/57 (33%)]	Source Labeling Loss: 2.3099	Source Domain Loss: 1.8599	Target Domain Loss: 0.1758
Epoch: 1/10	[Iter: 38/57 (66%)]	Source Labeling Loss: 2.1329	Source Domain Loss: 1.9768	Target Domain Loss: 0.1471
*****Source Accuracy: 0.3654
*****Target Accuracy: 0.2770 

model saved to p3_svhn->usps_best.pth
Epoch: 2/10	[Iter: 0/57 (0%)]	Source Labeling Loss: 1.8176	Source Domain Loss: 1.2558	Target Domain Loss: 0.3369
Epoch: 2/10	[Iter: 19/57 (33%)]	Source Labeling Loss: 1.5911	Source Domain Loss: 0.9768	Target Domain Loss: 0.4649
Epoch: 2/10	[Iter: 38/57 (66%)]	Source Labeling Loss: 1.5081	Source Domain Loss: 0.9304	Target Domain Loss: 0.4929
*****Source Accuracy: 0.6040
*****Target Accuracy: 0.4544 

model saved to p3_svhn->usps_best.pth
Epoch: 3/10	[Iter: 0/57 (0%)]	Source Labeling Loss: 1.2513	Source Domain Loss: 0.8488	Target Domain Loss: 0.5404
Epoch

In [8]:
def train_source(filename):
    source_train, source_test = source
    best_acc, n_epoch = 0, 10
    for epoch in range(n_epoch):
        for batch_idx, (img, label) in enumerate(source_train):
            img, label = img.to(device), label.to(device)
            my_net.zero_grad()
            output, _ = my_net(img)
            loss = loss_class(output, label)
            loss.backward()
            optimizer.step()
            if batch_idx % max(1, int(len(source_train)/3)) == 0:
                print('Epoch: {}/{}\t[Iter: {}/{} ({}%)]\tLoss: {:.4f}'.format(epoch+1, n_epoch, batch_idx,
                      len(source_train), int(batch_idx*100/len(source_train)), loss.item()))
        acc = test(source_test)
        print('*****Accuracy: {:.4f}'.format(acc),'\n')
        if acc > best_acc:
            best_acc = acc
            th.saveModel(filename+'_best.pth', my_net, optimizer)
    th.saveModel(filename+'_final.pth', my_net, optimizer)

In [16]:
# train single data
source = svhn_train_loader, svhn_test_loader
train_source('p3_svhn')

Epoch: 1/10	[Iter: 0/573 (0%)]	Loss: 0.6824
Epoch: 1/10	[Iter: 191/573 (33%)]	Loss: 0.6388
Epoch: 1/10	[Iter: 382/573 (66%)]	Loss: 0.4617
*****Accuracy: 0.8486 

model saved to p3_svhn_best.pth
Epoch: 2/10	[Iter: 0/573 (0%)]	Loss: 0.5575
Epoch: 2/10	[Iter: 191/573 (33%)]	Loss: 0.4899
Epoch: 2/10	[Iter: 382/573 (66%)]	Loss: 0.6138
*****Accuracy: 0.8648 

model saved to p3_svhn_best.pth
Epoch: 3/10	[Iter: 0/573 (0%)]	Loss: 0.3561
Epoch: 3/10	[Iter: 191/573 (33%)]	Loss: 0.5474
Epoch: 3/10	[Iter: 382/573 (66%)]	Loss: 0.4728
*****Accuracy: 0.8790 

model saved to p3_svhn_best.pth
Epoch: 4/10	[Iter: 0/573 (0%)]	Loss: 0.4416
Epoch: 4/10	[Iter: 191/573 (33%)]	Loss: 0.5590
Epoch: 4/10	[Iter: 382/573 (66%)]	Loss: 0.4797
*****Accuracy: 0.8775 

Epoch: 5/10	[Iter: 0/573 (0%)]	Loss: 0.4440
Epoch: 5/10	[Iter: 191/573 (33%)]	Loss: 0.4531
Epoch: 5/10	[Iter: 382/573 (66%)]	Loss: 0.5110
*****Accuracy: 0.8835 

model saved to p3_svhn_best.pth
Epoch: 6/10	[Iter: 0/573 (0%)]	Loss: 0.4763
Epoch: 6/10	[Iter:

In [17]:
# train single data
source = usps_train_loader, usps_test_loader
train_source('p3_usps')

Epoch: 1/10	[Iter: 0/57 (0%)]	Loss: 1.5498
Epoch: 1/10	[Iter: 19/57 (33%)]	Loss: 0.6240
Epoch: 1/10	[Iter: 38/57 (66%)]	Loss: 0.2379
*****Accuracy: 0.9332 

model saved to p3_usps_best.pth
Epoch: 2/10	[Iter: 0/57 (0%)]	Loss: 0.3653
Epoch: 2/10	[Iter: 19/57 (33%)]	Loss: 0.2053
Epoch: 2/10	[Iter: 38/57 (66%)]	Loss: 0.1492
*****Accuracy: 0.9432 

model saved to p3_usps_best.pth
Epoch: 3/10	[Iter: 0/57 (0%)]	Loss: 0.1634
Epoch: 3/10	[Iter: 19/57 (33%)]	Loss: 0.0854
Epoch: 3/10	[Iter: 38/57 (66%)]	Loss: 0.1495
*****Accuracy: 0.9532 

model saved to p3_usps_best.pth
Epoch: 4/10	[Iter: 0/57 (0%)]	Loss: 0.1401
Epoch: 4/10	[Iter: 19/57 (33%)]	Loss: 0.1205
Epoch: 4/10	[Iter: 38/57 (66%)]	Loss: 0.1254
*****Accuracy: 0.9562 

model saved to p3_usps_best.pth
Epoch: 5/10	[Iter: 0/57 (0%)]	Loss: 0.1301
Epoch: 5/10	[Iter: 19/57 (33%)]	Loss: 0.0911
Epoch: 5/10	[Iter: 38/57 (66%)]	Loss: 0.1073
*****Accuracy: 0.9557 

Epoch: 6/10	[Iter: 0/57 (0%)]	Loss: 0.2054
Epoch: 6/10	[Iter: 19/57 (33%)]	Loss: 0.0724

In [9]:
# train single data
source = mnist_train_loader, mnist_test_loader
train_source('p3_mnist')

Epoch: 1/10	[Iter: 0/469 (0%)]	Loss: 2.2860
Epoch: 1/10	[Iter: 156/469 (33%)]	Loss: 0.7390
Epoch: 1/10	[Iter: 312/469 (66%)]	Loss: 0.4562
Epoch: 1/10	[Iter: 468/469 (99%)]	Loss: 0.4062
*****Accuracy: 0.9485 

model saved to p3_mnist_best.pth
Epoch: 2/10	[Iter: 0/469 (0%)]	Loss: 0.3421
Epoch: 2/10	[Iter: 156/469 (33%)]	Loss: 0.3540
Epoch: 2/10	[Iter: 312/469 (66%)]	Loss: 0.4692
Epoch: 2/10	[Iter: 468/469 (99%)]	Loss: 0.2766
*****Accuracy: 0.9465 

Epoch: 3/10	[Iter: 0/469 (0%)]	Loss: 0.1642
Epoch: 3/10	[Iter: 156/469 (33%)]	Loss: 0.3209
Epoch: 3/10	[Iter: 312/469 (66%)]	Loss: 0.2668
Epoch: 3/10	[Iter: 468/469 (99%)]	Loss: 0.2371
*****Accuracy: 0.9645 

model saved to p3_mnist_best.pth
Epoch: 4/10	[Iter: 0/469 (0%)]	Loss: 0.2166
Epoch: 4/10	[Iter: 156/469 (33%)]	Loss: 0.1679
Epoch: 4/10	[Iter: 312/469 (66%)]	Loss: 0.2438
Epoch: 4/10	[Iter: 468/469 (99%)]	Loss: 0.1623
*****Accuracy: 0.9654 

model saved to p3_mnist_best.pth
Epoch: 5/10	[Iter: 0/469 (0%)]	Loss: 0.2211
Epoch: 5/10	[Iter: 15